In [8]:
from smilesfeature import feature_generate
import pandas as pd

data = {'SMILES': ['CCO', 'C1=CC=CC=C1', 'CCC', 'CN']}
df = pd.DataFrame(data)

In [9]:
df

,SMILES
0,CCO
1,C1=CC=CC=C1
2,CCC
3,CN


In [10]:
df = feature_generate(df)

In [18]:
df

,SMILES,Molecule,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,fingerprint,3D_Coordinates,Mol2Vec,image_array,3D_Coordinates_pc1,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,CCO,<rdkit.Chem.rdchem.Mol object at 0x0000023B24D...,1,1,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[0.9331702205988228, 0.050343946491454496, 0....","[1.3887247182428837, 0.22941507631912827, -1.0...","[[[255, 255, 255], [255, 255, 255], [255, 255,...",7.401487e-17,...,0,0,0,0,0,0,0,0,0,0
1,C1=CC=CC=C1,<rdkit.Chem.rdchem.Mol object at 0x0000023B24B...,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[-1.3083403284475346, -0.4011196171663065, 0....","[-0.7897782661020756, 0.44860777258872986, -1....","[[[255, 255, 255], [255, 255, 255], [255, 255,...",0.000000e+00,...,0,0,0,0,0,0,0,0,0,0
2,CCC,<rdkit.Chem.rdchem.Mol object at 0x0000023B24B...,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[1.1920961477330014, -0.29800235439020206, -0...","[2.5197734385728836, 0.09126146044582129, -0.7...","[[[255, 255, 255], [255, 255, 255], [255, 255,...",8.074349e-17,...,0,0,0,0,0,0,0,0,0,0
3,CN,<rdkit.Chem.rdchem.Mol object at 0x0000023B24B...,1,1,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[-0.5730904484168291, 0.029788530838895468, 0...","[0.33254989981651306, -0.2635647989809513, 0.0...","[[[255, 255, 255], [255, 255, 255], [255, 255,...",6.344132e-17,...,0,0,0,0,0,0,0,0,0,0


In [12]:
df.columns

Index(['SMILES', 'Molecule', 'Num_H_Acceptors', 'Num_H_Donors',
       'Num_RotatableBonds', 'fingerprint', '3D_Coordinates', 'Mol2Vec',
       'image_array', '3D_Coordinates_pc1',
       ...
       'fr_sulfide', 'fr_sulfonamd', 'fr_sulfone', 'fr_term_acetylene',
       'fr_tetrazole', 'fr_thiazole', 'fr_thiocyan', 'fr_thiophene',
       'fr_unbrch_alkane', 'fr_urea'],
      dtype='object', length=281)